<a href="https://colab.research.google.com/github/LucasLessa1/X-rayBodyPart/blob/davi1/C%C3%B3pia_de_Proj_CIS_Imagens_Raio_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Ajusting file


Proj_CIS_Imagens_Raio_X.ipynb

In [1]:
#A instalação da biblioteca pydicom 
#Ela trata da leitura de arquivos de imagens de exames de Raio-X, que são feitos em formatos DCM 

%pip install pydicom

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\davic\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
###########################################################################
#           Importanção de todos os pacotes utilizados                    # 
###########################################################################

import cv2            #Tratamento de imagem 
import pydicom         #Tratamento de arquivo DCM 
import pandas as pd     #Préprocessamento em tabelas
import os                #Pacote de tarefas de sistema operavional
import shutil            #Complementar à anterior
import time               #Medir tempo de execução 
import random              #Pacote para escolha de números de forma pseudo-aleatória 
import math                #Matemática 
import gdown                #Download de direitório salvo no Google Drive 
import zipfile               #Tratamento de arquivos em pasta zipadas
import numpy as np            #Operações numéricas e matemáticas em 'arrays'
from scipy.io import loadmat 
from matplotlib import pyplot     
import matplotlib.pyplot as plt   #Visualização de arquivo 
#from google.colab.patches import cv2_imshow    #Visualição de arquivo 
from tqdm import tqdm     #Barra de progresso
from PIL import Image     #Operações com imagens 
import string             
import glob             #Pacote de operações com múltiplos arquivos que possuam nome ou extensão em comum


from numpy import expand_dims    #Expansão de dimensões do "shape" (formato) da imagem
import tensorflow as tf          #Operações de vetores para o modelo de aprendizado profundo 
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
###########################################################################
#           Download e Extração dos arquivos originais                    # 
###########################################################################

#Funções:
 
def download(id): 
    url = 'https://drive.google.com/uc?id=' + str(id)
    gdown.download(url, output = None, quiet = False)

def unzip(path): #Função para unzip
    zip = zipfile.ZipFile(path)
    zip.extractall()
    zip.close()

In [39]:
#Chamando funções de Download e Extração 

# https://drive.google.com/file/d/1ev-r31j8oRzDlKM_toaeADO2psrA_XXm/view?usp=sharing

root_path = 'C:/Users/davic/OneDrive/Imagens/Documentos/Repositorio/X-rayBodyPart/X-rayBodyPart/'

# https://drive.google.com/file/d/1ev-r31j8oRzDlKM_toaeADO2psrA_XXm/view?usp=sharing

if os.path.isfile(f'{root_path}archive.zip'):
    print('O arquivo existe.')
    unzip(f'{root_path}archive.zip')    

else:
    download('1ev-r31j8oRzDlKM_toaeADO2psrA_XXm')
    unzip(f'{root_path}archive.zip')    


O arquivo existe.


In [24]:
#leitura dos arquivos originais para obter tanto o dataset de treino quanto o dataset de teste, que servirá de prova real

# os.chdir(f'{root_path}/drive/MyDrive/RAIO-X/RAIO-X/archive.zip (Unzipped Files)")
path = os.getcwd() 
train_df = pd.read_csv(f'{root_path}train.csv')
test_df = pd.read_csv(os.path.join(f'{root_path}sample_submission.csv'))

In [43]:
train_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3


#Pre-processing

In [26]:
#Confere propriedades de formato e conteudo do Pandas Dataset 
print("Columns:")
print(train_df.columns, "\n")

print("Types in columns:")
print(train_df.dtypes, "\n")

print("Types in info:")
print(train_df.describe(), "\n")

Columns:
Index(['SOPInstanceUID', 'Target'], dtype='object') 

Types in columns:
SOPInstanceUID    object
Target            object
dtype: object 

Types in info:
                                           SOPInstanceUID Target
count                                                1738   1738
unique                                               1738     41
top     1.2.826.0.1.3680043.8.498.10025629581362719970...     3 
freq                                                    1    724 



In [27]:
#Distribuição das labels em formato de dicionário com a chave sendo número e o valor sendo o nome da label em texto 

bodyparts = {
0 : 'Abdomen' ,
1 :'Ankle' ,
2 :'Cervical Spine',
3 : 'Chest' ,
4 :'Clavicles' ,
5 :'Elbow' ,
6 :'Feet' ,
7 : 'Finger' ,
8 : 'Forearm' ,
9 : 'Hand' ,
10 : 'Hip' ,
11 : 'Knee' ,
12 : 'Lower Leg' ,
13 : 'Lumbar Spine' ,
14 : 'Others' ,
15 :'Pelvis',
16 :'Shoulder' ,
17 :'Sinus' ,
18 : 'Skull' ,
19 : 'Thigh' ,
20 :'Thoracic Spine',
21: 'Wrist',
}

##Fixing folders

In [28]:
info = []

In [84]:
#Classe de processamento de imagem que mexe com pastas também -começa declarando variaveis de modificação de imagem 
class Image_processing():
  def __init__(self, folder, newFolder):
    self.fileList = []
    self.rotationRange = 90 
    self.brightRange = [0.2, 1.5]
    self.folder = folder #This is the folder of the original folder image
    self.newFolder = newFolder #This is the new folder

#constroi pastas para cada label
  def createfolders(self):
    os.mkdir(os.path.join(root_path, self.newFolder))
    #os.chdir(f'/content/{self.newFolder}') 
    for bodypart in list(bodyparts.values()):
      if os.path.isdir(os.path.join(root_path,self.newFolder, bodypart)) == False:
        os.mkdir(os.path.join(root_path,self.newFolder,bodypart))
      else:  
          pass       
#Coloca cada imagem do arquivo original "train/train/train" em uma pasta especifica de cada label para fins de treino do modelo 
  def folderImagebyLabel(self, df, dictionary):
    filelist = []
    #anda por todos os diretorios e arquivos de "train/train/train"
    for root, dirs, files in os.walk(f'{root_path}{str(self.folder)}'):
      for file in files:
        filelist.append(os.path.join(root,file))

    for filename in filelist:
      #Lê os arquivos de formato DCM por meio da biblioteca apropriada
      dicom = pydicom.dcmread(filename)
      #Transforma a imagem em numpy array 
      img = dicom.pixel_array
      #Separa seu nome por cada barra - o nome sendo identificador_de_estudo/identificador_de_serie_de_exames/identificador_unico_de_imagem 
      var = filename.split("\\")
      #Pega o último termo que foi separado - identificador unico
      var = var[-1]
      #Vai atrás do arquivo no dataframe formado para econtrar sua label específica; omiti deste identficador o "-c.dcm"(extensão do arquivo)
      row = df.index[df['SOPInstanceUID']==f'{var[:-6]}'].tolist()
      #Guarda a label
      target = df['Target'].iloc[row[0]]
      #Aqui nós arbitrariamente não utilizamos imagens com mais de uma label para não prejudicar o aprendizado do modelo
      if len(target.strip()) > 2: 
        continue
      #Encontra o nome da label baseada no número da coluna do dataframe conferindo no dicionário já feito
      label = dictionary.get(int(target))
      #Passo importante de formatar a imagem numa amplitude de pixels comumente aceita nas funções de processamento de imagem 
      resized_img  = (np.maximum(img,0)/img.max())*255 
      #Transforma em formato 8 bits pelo mesmo motivo 
      im = resized_img.astype(np.uint8)
      # Chama classe CLAHE(equaliador de brilho) -argumentos: valor máximo para limitação do contraste(evitar predominancia de sinal ruim), tamanho dos grides de divisão de imagem para se aplicar o processo
      clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
      # Aplica no canal que importa- a luminosidade
      clahe_img = clahe.apply(im)
      #Volta a colocá-lo em numpy array 
      im = Image.fromarray(clahe_img)
      #Salva imagem em formato PNG 
      im.save(os.path.join(f'{root_path}{self.newFolder}', f'{label}', f'{var[:-6]}.png'))

  #Este método coloca todas as imagens do diretório "train/train/train" misturada em um pasta só para fins de testagem do modelo (mesmo processo)
  def allImagesFolder(self, df):
    filelist = []
    for root, dirs, files in os.walk(f'{root_path}{str(self.folder)}'):
      for file in files:
        filelist.append(os.path.join(root,file))

    for filename in filelist:
      dicom = pydicom.dcmread(filename)
      img = dicom.pixel_array
      var = filename.split("/")
      var = var[-1]
      row = df.index[df['SOPInstanceUID']==f'{var[:-6]}'].tolist()
      resized_img  = (np.maximum(img,0)/img.max())*255 
      im = resized_img.astype(np.uint8)
      clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
      clahe_img = clahe.apply(im)
      im = Image.fromarray(clahe_img)
      im.save(os.path.join(f'{root_path}{self.newFolder}/', f'{var[:-6]}.png'))


 

In [85]:
print(train_df.index[train_df['SOPInstanceUID']==f'{var[:-6]}'].tolist())

NameError: name 'var' is not defined

In [86]:
print(train_df['Target'].iloc[row[0]])

NameError: name 'row' is not defined

In [87]:
print(root_path)

C:/Users/davic/OneDrive/Imagens/Documentos/Repositorio/X-rayBodyPart/X-rayBodyPart/


In [88]:
#Organize the test images folder
nameFolderTest = "test_img"
processImageTest = Image_processing(folder='test', newFolder = nameFolderTest)

try:
  os.mkdir(f'{root_path}{nameFolderTest}')
  processImageTest.allImagesFolder(train_df)

except:
  shutil.rmtree(f'{root_path}{nameFolderTest}')
  os.mkdir(f'{root_path}{nameFolderTest}')
  processImageTest.allImagesFolder(train_df)
print('')

In [89]:
#Organize the train images folder
nameFolderTrain = "train_img"
processImageTrain = Image_processing(folder='train', newFolder = nameFolderTrain)

try:
  processImageTrain.createfolders()
  #This "train" is the name of the original folder of the images
  processImageTrain.folderImagebyLabel(train_df, bodyparts)
except:
  shutil.rmtree(f'{root_path}{nameFolderTrain}')
  processImageTrain.createfolders()
  processImageTrain.folderImagebyLabel(train_df, bodyparts)


In [90]:
#Classe do data augmentation propriamente dito que vai herdar as propriedades da classe anterior 
class Data_Augumentation(Image_processing):

  def __init__(self, folder, newFolder, folder_img):
    #super().__init__() chama as propriedades da classe superior ao colocar os argumentos necessários para rodar aquela classe
    super().__init__(folder, newFolder)
    # Image_processing.__init__(self, folder, newFolder)
    self.folder_img = folder_img

 #Este método cria imagens artificias baseada na necessidade de mais imagens a fim de se ter o mesmo número de imagens requisitadas em cada label
  def process_imgs(self, qntd = 100):
    for folder in os.listdir(os.path.join(root_path, self.folder_img)):
      #real_qntd é a quantidade de imagens de exames reais
      real_qntd = len(os.listdir(os.path.join(root_path, self.folder_img, folder)))
      diff = qntd - real_qntd
      #Aqui se compara se temos mais imagens do que precisamos ou menos 
      if qntd > real_qntd: 
        #A quantidade que temos tem que ser maior do que a diferença
        #Porque num fator de uma imagem artificial para uma original conseguimos apenas dobrar a quantidade original 
        if real_qntd >= diff:
          #exams  = lista de arquivos sorteados de imagens para cara pasta de label 
          exams = random.choices(os.listdir(os.path.join(root_path, self.folder_img, folder)), k= diff)
          self.augment(folder, exams, 1)
        else:
          #Precisamos de imagens adicionais em quantas vezes mais do que já temos?
          for i in range(100):
            check = real_qntd + real_qntd*i
            if check > qntd: 
              break
            multiplier = i
            diff_mult = qntd - check
            
          #De cada uma imagem original vamos pegar x vezes para aproximar do requerido 
          exams = [x for x in os.listdir(os.path.join(root_path, self.folder_img, folder)) if x.find("copy") == -1 ]
          self.augment(folder, exams, multiplier)
          #Aqui pegamos por sorteio o resto que escapa do multiplo
          if diff_mult>0:
            exams = [x for x in os.listdir(os.path.join(root_path, self.folder_img, folder)) if x.find("copy") == -1 ]
            exams = random.choices(exams, k= diff_mult)
            self.augment(folder, exams, 1)
          
          
      #Caso se tenha mais imagens originais do que o requisitado, nos desfazemos das imagens em excesso por meio de sorteio também
      elif real_qntd > qntd:
        diff = (real_qntd-qntd)
        sorteados = glob.glob(f'{os.path.join(root_path, self.folder_img, folder)}/*')
        for s in random.sample(sorteados, diff):
          os.remove(s)


  def augment(self, folder, exams, limit):
    #folder = nome que eu dei para pasta da label (ex : skull)
    #exams  = lista de arquivos de imagens para cara pasta de label 
    #limit = quantidade de imagens artificias criadas a partir da original
    
    nameImage = []
    cont1 = -1
    for exam in exams: 
      #Transfere os elementos de uma lista para outra para fins de organização
      nameImage.append(exam)
      #Carrega a imagem 
      img = load_img(os.path.join(root_path, self.folder_img, folder, exam))
      #Transforma em numpy array 
      data = img_to_array(img)
      #Acrescenta uma dimenção no "shape" da imagem referente ao batch - que será tamanho 1, o que não muda nada e serve apenas para rodar a função
      samples = expand_dims(data, 0)
      #Chama gerador de imagens utilizando dos argumentos de alteração de imagens - superficiais, mas o suficiente para confundir o modelo de aprendizado...
      #... e semelhante o suficiente para não desfazer os formatos essenciais de partes do corpo humano que serão analisadas 
      datagen = ImageDataGenerator(horizontal_flip=True,rotation_range = self.rotationRange, brightness_range = self.brightRange)
      #Prepara o iterador 
      it = datagen.flow(samples, batch_size=1)
      cont1 =+ 1
      #Esta variável é uma "compreensão de lista" que verifica se a imagem não é uma duplicata ao olhar a lista "nameImage", que serve de histórico
      dup = [x for i, x in enumerate(nameImage) if i != nameImage.index(x)]

      #Em caso de duplicata, é colocado é escrito em seu nome que trata-se de repetição e um identificador aleatório 
      if True:
        equalDup = "_repeted_" + str(random.randint(-9999,9999))
      else:
        equalDup = ""
        
      for cont in range(limit):
        # Aqui trata-se do processo de adequação da imagem nos formatos necessários e salvá-la com nome único para evitar sobrescrição dos arquivos
        batch = it.next()
        image = batch[0].astype('uint16')
        resized_img  = (np.maximum(image,0)/image.max())*255 
        im = Image.fromarray(resized_img.astype(np.uint8))
        rand = random.randint(-9999,9999)
        info.append(f'{os.path.join(root_path, self.folder_img, folder, exam[:-4])}_copy_{cont1}_{cont}.png')
        im.save(f'{os.path.join(root_path, self.folder_img, folder, exam[:-4])}_{equalDup}_copy_{cont1}_{cont}.png')


In [91]:
#Augumented for Train image

#Esta função ja pode ser chamada sem muita discrição por conseguir se repetir sem deixar rastros, diferentes das prévias que mexem com pastas
AugumentedTrain  =  Data_Augumentation(folder='train', newFolder = nameFolderTrain, folder_img = "train_img")
AugumentedTrain.process_imgs()

# #Augumented for Train image
# AugumentedTest  =  Data_Augumentation(folder='test', newFolder = nameFolderTest, folder_img = "test_img")
# AugumentedTest.process_imgs()

In [93]:
for files in os.listdir(os.path.join(root_path,'train_img')):
  print(files)
  print(len(os.listdir(os.path.join(root_path, 'train_img', files))))

Abdomen
100
Ankle
100
Cervical Spine
100
Chest
100
Clavicles
100
Elbow
100
Feet
100
Finger
100
Forearm
100
Hand
100
Hip
100
Knee
100
Lower Leg
100
Lumbar Spine
100
Others
100
Pelvis
100
Shoulder
100
Sinus
100
Skull
100
Thigh
100
Thoracic Spine
100
Wrist
100


In [96]:
#Esta classe irá criar outro diretório para todos os arquivos serem colocados juntos 
class SelectFolders():
#Declaram-se as variáveis iniciais baseados em caminhos de pasta e quantidade de imagens requeridas por input 
  def __init__(self, mainPath, root, qntd):
    self.mainPath = mainPath
    self.qntdImages = qntd
    self.listNames = []
    self.flag = []
    self.rootPath = root
  
  #Verifica a quantidade de cada arquivo em cada pasta 
  def verify(self, folder2verify, order):
    for files in os.listdir(folder2verify):
      self.listNames.append([files, len(os.listdir(f'{folder2verify}/{files}')) ])
      #Verifica lista listNames que possui os nomes das pastas de labels e o numeros de arquivos nelas

    for i in self.listNames:
      if i[1] > self.qntdImages:
        #False se tiver pastas com menos de 100 imagens ou menos que self.qntdImages
        #A lista flag quarda o valor de falos, e em seguida o nome da pasta da label e o numero de imagens nele 
        self.flag.append([False, i[0], i[1]])
    #Printa a lista se requisitado 
    if order == "print":
      print(self.flag)
    if order =="result":
      flagzinha = False
      #zip faz uma tupla de tuplas de todos o primeiros, segundos e terceiros elementos 
      for i in zip(self.flag): 
        if i[0] ==True:
          print(f"Error: The folder {i[1]} has {i[2]} Images.")
        else:
          flagzinha = True

      if flagzinha == True: 
        print(f"Every folder has more than {self.qntdImages} Images.")

  #Este metódo constrói pasta uma pasta com pasta de labels a semelhança do que já foi feito 
  #Mas antes será verificado se isso já não foi feito 
  def folder2Model(self, path, qntd , bodyparts): 
    try:
      os.mkdir(os.path.join(self.rootPath, path)) 
      print(self.rootPath)
      print(path)
    except:
      shutil.rmtree(os.path.join(root_path, path))
      os.mkdir(os.path.join(self.rootPath,path))

    #Para cada label em formato de texto é verificado se já existe uma pasta
    for part in list(bodyparts.values()):
      if os.path.isdir(os.path.join(self.rootPath, path , part)) == False:
        os.mkdir(os.path.join(self.rootPath, path, part))
      #Seleciona 'x' imagens de uma pasta de label e copia para imageModel
      selectImages = random.choices(os.listdir(os.path.join(self.mainPath, part)), k= qntd)

      for image in selectImages:
        pathImageModel = os.path.join(self.rootPath, path, part)
        originalPathImages = os.path.join(self.mainPath, part, image)
        # print(originalPathImages)
        shutil.copy(originalPathImages, pathImageModel)


In [97]:
fix2 = SelectFolders(os.path.join(root_path, 'train_img'), root_path ,  100)
path = "imageModel/"
#Verify if train_img has more than 100 images 
fix2.verify( os.path.join(root_path, 'train_img') ,"result")

#Organizing the folder for the model
fix2.folder2Model(path, 100, bodyparts)

#Verify if imageModel has 100 images for each folder 
print()
fix2.verify(os.path.join(root_path, 'imageModel') ,"result")

C:/Users/davic/OneDrive/Imagens/Documentos/Repositorio/X-rayBodyPart/X-rayBodyPart/
imageModel/



# Model

In [100]:
#Selecionando tamanho da template
batch_size = 32  
img_height = 180 
img_width = 180

data_dir = os.path.join(root_path, "train_img") 

In [101]:
#Constrói um tensorflow dataset para poder ser colocado no modelo da biblioteca Keras 
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode = 'categorical',
  class_names = list(bodyparts.values()),
  validation_split=0.2, #Parte destinada aum dataset de validação 
  subset="training",
  seed=123, #senha da randomização do processo 
  image_size=(img_height, img_width), #"shape" do arquivo
  batch_size=batch_size)


Found 2200 files belonging to 22 classes.
Using 1760 files for training.


In [102]:
list(bodyparts.values())

['Abdomen',
 'Ankle',
 'Cervical Spine',
 'Chest',
 'Clavicles',
 'Elbow',
 'Feet',
 'Finger',
 'Forearm',
 'Hand',
 'Hip',
 'Knee',
 'Lower Leg',
 'Lumbar Spine',
 'Others',
 'Pelvis',
 'Shoulder',
 'Sinus',
 'Skull',
 'Thigh',
 'Thoracic Spine',
 'Wrist']

In [103]:
#Constrói um tensorflow dataset para poder ser colocado no modelo da biblioteca Keras 
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode = 'categorical',
  class_names = list(bodyparts.values()),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2200 files belonging to 22 classes.
Using 440 files for validation.


In [105]:
for files in os.listdir(os.path.join(root_path, 'train_img')):
  print(files)
  print(len(os.listdir(os.path.join(root_path, 'train_img', files))))

Abdomen
100
Ankle
100
Cervical Spine
100
Chest
100
Clavicles
100
Elbow
100
Feet
100
Finger
100
Forearm
100
Hand
100
Hip
100
Knee
100
Lower Leg
100
Lumbar Spine
100
Others
100
Pelvis
100
Shoulder
100
Sinus
100
Skull
100
Thigh
100
Thoracic Spine
100
Wrist
100


## Extra

In [106]:
class_names = train_ds.class_names
#print(class_names)

In [108]:
#Conferindo "shape" - ou formato 
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32, 22)


In [109]:
#Conferindo "shape" - ou formato 
for image_batch, labels_batch in val_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32, 22)


## Implementing Model

In [110]:
import tensorflow.keras as K

In [116]:
#This function belongs to Matheus Vieira, current IEEE-CIS Project Director
def plot_history(history: K.callbacks.History) -> None:
  #Show model loss and accuracy while training 
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [111]:
input_m = K.Input(shape=(180, 180, 3))
res_model = K.applications.ResNet50(include_top=False,
                                    weights="imagenet",
                                    input_tensor=input_m)

94765736/94765736 [==============================] - 9s 0us/step


In [112]:
for layer in res_model.layers[:143]:
  layer.trainable = False

In [113]:
for i, layer in enumerate(res_model.layers):
  print(i, layer.name, "-", layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

In [114]:
model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(512, activation='relu'))
model.add(K.layers.Dropout(0.75))
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layeers.Bach)
model.add(K.layers.Dense(22, activation='softmax'))

#Testando para 100 imagens 

In [109]:
check_point1 = K.callbacks.ModelCheckpoint(filepath="checkpoint_restnet50_100img.h5",
                                          monitor="val_acc",
                                          mode="max",
                                          save_best_only=True)

In [110]:
model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

history1 = model.fit(train_ds, batch_size=32, epochs=50, verbose=1,
                    validation_data=val_ds,
                    callbacks=[check_point1])

model.summary()
model.save('modelo_resnet50_100img.h5')

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


55/55 [==============================] - ETA: 0s - loss: 3.4956 - accuracy: 0.0841

55/55 [==============================] - 439s 8s/step - loss: 3.4956 - accuracy: 0.0841 - val_loss: 2.8338 - val_accuracy: 0.1841
Epoch 2/20
55/55 [==============================] - ETA: 0s - loss: 2.9220 - accuracy: 0.1381

55/55 [==============================] - 425s 8s/step - loss: 2.9220 - accuracy: 0.1381 - val_loss: 2.5696 - val_accuracy: 0.3227
Epoch 3/20
55/55 [==============================] - ETA: 0s - loss: 2.7239 - accuracy: 0.1915

...

55/55 [==============================] - 416s 8s/step - loss: 1.3099 - accuracy: 0.5869 - val_loss: 0.8178 - val_accuracy: 0.8136
Epoch 17/20
55/55 [==============================] - ETA: 0s - loss: 1.3477 - accuracy: 0.5778

55/55 [==============================] - 419s 8s/step - loss: 1.3477 - accuracy: 0.5778 - val_loss: 0.8239 - val_accuracy: 0.8068
Epoch 18/20
55/55 [==============================] - ETA: 0s - loss: 1.3030 - accuracy: 0.6006

55/55 [==============================] - 415s 8s/step - loss: 1.3030 - accuracy: 0.6006 - val_loss: 0.8092 - val_accuracy: 0.8045
Epoch 19/20
55/55 [==============================] - ETA: 0s - loss: 1.2341 - accuracy: 0.6335

55/55 [==============================] - 409s 7s/step - loss: 1.2341 - accuracy: 0.6335 - val_loss: 0.7725 - val_accuracy: 0.8227
Epoch 20/20
55/55 [==============================] - ETA: 0s - loss: 1.1885 - accuracy: 0.6330

55/55 [==============================] - 420s 8s/step - loss: 1.1885 - accuracy: 0.6330 - val_loss: 0.7285 - val_accuracy: 0.8409
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 6, 6, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 73728)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               9437312   
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0

In [ ]:
plot_history(history1)

### Testando para 200 imagens 

In [ ]:
fix2 = SelectFolders(os.path.join(root_path, 'train_img'), root_path ,  200)
path = "imageModel/"
#Verify if train_img has more than 100 images 
fix2.verify( os.path.join(root_path, 'train_img') ,"result")

#Organizing the folder for the model
fix2.folder2Model(path, 200, bodyparts)

#Verify if imageModel has 100 images for each folder 
print()
fix2.verify(os.path.join(root_path, 'imageModel') ,"result")

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode = 'categorical',
  class_names = list(bodyparts.values()),
  validation_split=0.2, #Parte destinada aum dataset de validação 
  subset="training",
  seed=123, #senha da randomização do processo 
  image_size=(img_height, img_width), #"shape" do arquivo
  batch_size=batch_size)


In [ ]:
#Constrói um tensorflow dataset para poder ser colocado no modelo da biblioteca Keras 
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode = 'categorical',
  class_names = list(bodyparts.values()),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
check_point2 = K.callbacks.ModelCheckpoint(filepath="checkpoint_restnet50_200img.h5",
                                          monitor="val_acc",
                                          mode="max",
                                          save_best_only=True)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

history2 = model.fit(train_ds, batch_size=32, epochs=50, verbose=1,
                    validation_data=val_ds,
                    callbacks=[check_point2])

model.summary()
model.save('modelo_resnet50_200img.h5')

In [ ]:
plot_history(history2)

# Testando para 300 imagens 

In [ ]:
fix2 = SelectFolders(os.path.join(root_path, 'train_img'), root_path ,  300)
path = "imageModel/"
#Verify if train_img has more than 100 images 
fix2.verify( os.path.join(root_path, 'train_img') ,"result")

#Organizing the folder for the model
fix2.folder2Model(path, 300, bodyparts)

#Verify if imageModel has 100 images for each folder 
print()
fix2.verify(os.path.join(root_path, 'imageModel') ,"result")

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode = 'categorical',
  class_names = list(bodyparts.values()),
  validation_split=0.2, #Parte destinada aum dataset de validação 
  subset="training",
  seed=123, #senha da randomização do processo 
  image_size=(img_height, img_width), #"shape" do arquivo
  batch_size=batch_size)


In [ ]:
#Constrói um tensorflow dataset para poder ser colocado no modelo da biblioteca Keras 
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  label_mode = 'categorical',
  class_names = list(bodyparts.values()),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
check_point3 = K.callbacks.ModelCheckpoint(filepath="checkpoint_restnet50_300img.h5",
                                          monitor="val_acc",
                                          mode="max",
                                          save_best_only=True)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=K.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

history3 = model.fit(train_ds, batch_size=32, epochs=50, verbose=1,
                    validation_data=val_ds,
                    callbacks=[check_point3])

model.summary()
model.save('modelo_resnet50_300img.h5')

In [ ]:
plot_history(history3)